**Use Case / Problem**

In online discussions, debates, and academic settings, people often struggle to present strong arguments or anticipate counterarguments effectively. This leads to poor persuasion, weak reasoning, and a lack of critical thinking in public discourse.

**How Generative AI Solves It**

This project demonstrates how Generative AI can help users:
- Strengthen their arguments,
- Identify possible counterpoints,
- Generate logical rebuttals.

Using few-shot prompting, structured JSON output, and AI-driven evaluation, the system acts as a personal debate coach. It takes a user opinion and returns a well-structured set of arguments, helping users reason more clearly and think critically. This empowers users—students, debaters, or professionals—to communicate ideas with greater confidence and precision.


In [1]:
!pip install -qU "google-genai>=1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.7/159.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 3.2 MB/s eta 0:00:00


Import necessary modules, including:
- `google.genai` for accessing GenAI models,
- `kaggle_secrets` to securely use API keys stored in Kaggle,
- `json` for parsing model responses.

In [2]:
from google import genai
from kaggle_secrets import UserSecretsClient
from google.genai import types
import json

user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

MODEL_ID = "models/gemini-2.0-flash-exp"

This function constructs a structured prompt with examples to help the AI understand how to strengthen an argument, provide a counterargument, and generate a rebuttal.

GenAI Capability:
- Few-shot prompting
- Structured output / JSON mode (model is explicitly asked to return JSON)


In [3]:
# Prompt with grounded context
def build_few_shot_prompt(user_opinion):
    example = """
Example:
User opinion: "Social media is harmful to mental health."

{
  "strengthened_argument": "Studies show increased social media use correlates with anxiety and depression...",
  "counter_argument": "Some argue social media fosters support networks...",
  "rebuttal": "Support exists, but the design prioritizes engagement over mental well-being."
}
"""
    return f"""
You are an AI debate assistant.

Respond ONLY in JSON with:
- strengthened_argument
- counter_argument
- rebuttal

{example}

Now analyze:
User opinion: "{user_opinion}"
"""


user_opinion = "Climate change is real."
prompt = build_few_shot_prompt(user_opinion)

config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=config_with_search,
)

Helper function to extract valid JSON objects from raw model responses. It ensures proper parsing and cleanup of formatted text before loading it as structured data.

GenAI Capability:
- Structured output / JSON mode


In [4]:
import json
import re

def extract_json_from_response(text):
    # Try to find a JSON code block first
    match = re.search(r"```json(.*?)```", text, re.DOTALL)
    if match:
        json_str = match.group(1).strip()
    else:
        # Fallback: find the first { ... } block
        match = re.search(r"{.*}", text, re.DOTALL)
        if match:
            json_str = match.group(0).strip()
        else:
            return None

    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return None

parsed = extract_json_from_response(response.text)

if parsed:
    print("User opinion: ", user_opinion)
    print("📢 Strengthened Argument:\n", parsed["strengthened_argument"])
    print("\n⚔️ Counter-Argument:\n", parsed["counter_argument"])
    print("\n🛡️ Rebuttal:\n", parsed["rebuttal"])
else:
    print("⚠️ Still couldn't parse response. Raw output:")
    print(response.text)

User opinion:  Climate change is real.
📢 Strengthened Argument:
 Multiple lines of evidence, including rising global temperatures, melting ice sheets and glaciers, and ocean acidification, demonstrate that Earth's climate is changing rapidly. The scientific consensus, with over 97% agreement, attributes this change primarily to human activities, particularly the emission of greenhouse gases.

⚔️ Counter-Argument:
 Some argue that climate change is a natural cycle, pointing to historical periods of warming and cooling. They may also claim that the impact of human emissions is negligible compared to natural factors, or that climate models are unreliable.

🛡️ Rebuttal:
 While natural climate variability exists, the current rate of warming is unprecedented in recent history and far exceeds what can be explained by natural factors alone. Climate models, though simplifications of a complex system, have accurately predicted many observed trends and are continually refined. The overwhelming sc

Creates an evaluation prompt to assess the generated arguments. It uses a rubric with metrics (Relevance, Logic, Clarity, etc.) to score model outputs on a scale of 1 to 5.

Capability:
- Gen AI evaluation (using the model to rate outputs)

In [5]:
# Evaluate the quality of the generated arguments
evaluation_prompt = f"""You are a critical thinking expert.
Evaluate the following debate arguments on a scale of 1 to 5 for:
- Relevance
- Logic
- Persuasiveness

Then provide brief feedback.

Input:
{json.dumps(parsed, indent=2)}

Respond in JSON:
{{
  "strengthened_argument_score": ...,
  "counter_argument_score": ...,
  "rebuttal_score": ...,
  "feedback": "..."
}}
"""

eval_response = client.models.generate_content(
    model=MODEL_ID,
    contents=evaluation_prompt
)

Cleans up raw evaluation text from the model, removing Markdown formatting like triple backticks and ensuring valid JSON is returned.

GenAI Capability:
- Structured output / JSON mode

In [6]:
# Clean and display eval response
eval_text = eval_response.text.strip()
if eval_text.startswith("```json"):
    eval_text = re.sub(r"^```json\s*", "", eval_text)
    eval_text = re.sub(r"```\s*$", "", eval_text)

try:
    eval_data = json.loads(eval_text)
    print("🧪 Evaluation Scores:")
    print(json.dumps(eval_data, indent=2))
except Exception as e:
    print("⚠️ Could not parse evaluation response:")
    print(eval_response.text)

🧪 Evaluation Scores:
{
  "strengthened_argument_score": {
    "Relevance": 5,
    "Logic": 5,
    "Persuasiveness": 5
  },
  "counter_argument_score": {
    "Relevance": 4,
    "Logic": 3,
    "Persuasiveness": 3
  },
  "rebuttal_score": {
    "Relevance": 5,
    "Logic": 5,
    "Persuasiveness": 5
  },
  "feedback": "The strengthened argument provides strong evidence and cites scientific consensus, making it very persuasive. The counter-argument raises relevant points, but lacks the robust evidence to back them up. The rebuttal effectively addresses the counter-argument's claims by highlighting the unprecedented rate of change, the validity of climate models, and the economic consequences of inaction. The rebuttal's use of 'overwhelming scientific consensus' is a rhetorical strength. Overall, the debate showcases a strong understanding of the issues."
}
